<a href="https://colab.research.google.com/github/kopaprin/shogi_viz/blob/master/shogi_taikyoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/shogi_pj/02_taikyoku")
os.getcwd() 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/content/drive/My Drive/shogi_pj/02_taikyoku'

In [0]:
!pip install requests

import sys
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
import urllib.robotparser

import numpy as np
import pandas as pd
from tqdm import tqdm



import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import time

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

def AngleBrackets(str_): # 山かっこ内だけ正規表現で抽出する
  result = re.search(r'>(.*?)<',str_).group(0)[1:][:-1]
  return result

!pip install fastprogress

from fastprogress.fastprogress import master_bar, progress_bar

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [835 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/univers

In [0]:
!pip install selenium
from pprint import pprint
from selenium import webdriver
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


In [0]:
import urllib.robotparser

robots_url = "https://crowdworks.jp/robots.txt"

# 1) RobotFileParserインスタンスの作成
rp = urllib.robotparser.RobotFileParser()
# 2) URLのセット
rp.set_url(robots_url)
# 3)robots.txtの読み込みと解析
rp.read()

In [0]:
url = "https://shogidb2.com/"

# (1-1)robotsから取得できるか確認
print('*: '+str(rp.can_fetch('*', url)))

# (1-2)robotsから取得できるか確認
print('baiduspider: '+ str(rp.can_fetch('baiduspider', url)))

# (2-1)クロール間隔の確認
print('*: '+str(rp.crawl_delay('*')))

# (2-2)クロール間隔の確認
print('bingbot: '+str(rp.crawl_delay('bingbot')))


*: True
baiduspider: False
*: None
bingbot: 10


In [0]:

url = "https://shogidb2.com/latest/page/1"
headers = {
    'User-Agent': 'kopaprin Crawler (kopaprin1125@gmail.com)'
    }
r = requests.get(url,headers)
r.encoding = r.apparent_encoding

soup = BeautifulSoup(r.text,"html.parser")
for n in soup.select("#app > div > div:nth-of-type(2) > div.col-12.col-md-6 > div > a:nth-of-type(1) > div > div.w-100 > p:nth-of-type(1)"):
  print(n)

In [0]:
with open('sample.html', 'w') as file:
    file.write(r.text)

In [0]:
# base_df初期化用コード

"""

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent = kopaprin Crawler (kopaprin1125@gmail.com)")

driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)
driver.get("https://shogidb2.com/latest/page/1")
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

# URL取得　OK
url_prefix = "https://shogidb2.com"

# output
game_id_list = []
url_list = []
img_list = []
game_date_list = []
game_title_list = []
black_list= []
#black_rank_list = []
white_list = []
#white_rank_list = []
senkei_list = []
teai_list = []

# game
for n in soup.select("#app > div > div > div > div > a"):
  game_id_list.append(re.search(r'games/(.*?)"',str(n)).group(0)[6:][:-1])
  url = re.search(r'href="(.*?)"',str(n)).group(0)[6:][:-1] #ok
  url_list.append(urllib.parse.urljoin(url_prefix,url))
  img_list.append(re.search(r'src="(.*?)"',str(n)).group(0)[5:][:-1])

  title = n.select("p")
  
  game_date = re.search(r'>(.*?)日',str(title[0])).group(0)[1:]
  game_date_list.append(datetime.datetime.strptime(game_date, '%Y年%m月%d日')) #ok
  game_title_list.append(re.search(r'日(.*?)<',str(title[0])).group(0)[1:][:-1]) #ok
  black_list.append(re.search(r'先手 (.*?)<',str(title[1])).group(0)[3:][:-1].split(" ")[0].replace("　","")) #ok
  #black_rank_list.append(re.search(r'先手 (.*?)<',str(title[1])).group(0)[3:][:-1].split(" ")[1]) #ok  
    
  white_list.append(re.search(r'後手 (.*?)<',str(title[2])).group(0)[3:][:-1].split(" ")[0].replace("　","")) #ok
  #white_rank_list.append(re.search(r'後手 (.*?)<',str(title[2])).group(0)[3:][:-1].split(" ")[1]) #ok  
  
  senkei_list.append(re.search(r':(.*?)<',str(title[3])).group(0)[2:][:-1])
  teai_list.append(re.search(r':(.*?)<',str(title[4])).group(0)[2:][:-1])

df = pd.DataFrame({
    "game_id":game_id_list
    ,"game_date":game_date_list
    ,"game_title":game_title_list
    ,"black":black_list
    #,"black_rank":black_rank_list
    ,"white":white_list
    #,"white_rank":white_rank_list
    ,"senkei":senkei_list
    ,"teai":teai_list
    ,"url":url_list
    ,"transaction":1

})
df.to_csv("shogi_taikyoku.csv",encoding="utf-8",header=True,index=False)

"""



'\n\noptions = webdriver.ChromeOptions()\noptions.add_argument(\'--headless\')\noptions.add_argument(\'--no-sandbox\')\noptions.add_argument(\'--disable-dev-shm-usage\')\noptions.add_argument("user-agent = kopaprin Crawler (kopaprin1125@gmail.com)")\n\ndriver = webdriver.Chrome(\'chromedriver\',options=options)\ndriver.implicitly_wait(10)\ndriver.get("https://shogidb2.com/latest/page/1")\nhtml = driver.page_source.encode(\'utf-8\')\nsoup = BeautifulSoup(html, "html.parser")\n\n# URL取得\u3000OK\nurl_prefix = "https://shogidb2.com"\n\n# output\ngame_id_list = []\nurl_list = []\nimg_list = []\ngame_date_list = []\ngame_title_list = []\nblack_list= []\n#black_rank_list = []\nwhite_list = []\n#white_rank_list = []\nsenkei_list = []\nteai_list = []\n\n# game\nfor n in soup.select("#app > div > div > div > div > a"):\n  game_id_list.append(re.search(r\'games/(.*?)"\',str(n)).group(0)[6:][:-1])\n  url = re.search(r\'href="(.*?)"\',str(n)).group(0)[6:][:-1] #ok\n  url_list.append(urllib.parse.ur

# ここからループ処理コード

In [0]:
#1 既存dfの読み込みと、game_idの取得

base_df = pd.read_csv("shogi_taikyoku.csv")



In [0]:
base_df.duplicated(subset="game_id").value_counts()

False    9000
dtype: int64

In [0]:
#2 サイトクローリング、game_id取得、内容判定、新規db作成

num = np.arange(1,50)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent = kopaprin Crawler (kopaprin1125@gmail.com)")

driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

for i in progress_bar(num) :
  get_url = "https://shogidb2.com/latest/page/" + str(i)
  time.sleep(2)
  driver.get(get_url)
  html = driver.page_source.encode('utf-8')
  soup = BeautifulSoup(html, "html.parser")


  # URL取得　OK
  url_prefix = "https://shogidb2.com"

  # output
  game_id_list = []
  url_list = []
  img_list = []
  game_date_list = []
  game_title_list = []
  black_list= []
  #black_rank_list = []
  white_list = []
  #white_rank_list = []
  senkei_list = []
  teai_list = []
  df = pd.DataFrame()

  # game
  for n in soup.select("#app > div > div > div > div > a"):
    game_id = re.search(r'games/(.*?)"',str(n)).group(0)[6:][:-1]
    acquired_id = list(base_df["game_id"].values)
    if game_id in acquired_id:
      pass
    else :
      game_id_list.append(game_id)
      url = re.search(r'href="(.*?)"',str(n)).group(0)[6:][:-1] #ok
      url_list.append(urllib.parse.urljoin(url_prefix,url))
      img_list.append(re.search(r'src="(.*?)"',str(n)).group(0)[5:][:-1])

      title = n.select("p")
      
      game_date = re.search(r'>(.*?)日',str(title[0])).group(0)[1:]
      game_date_list.append(datetime.datetime.strptime(game_date, '%Y年%m月%d日')) #ok
      game_title_list.append(re.search(r'日(.*?)<',str(title[0])).group(0)[1:][:-1]) #ok
      black_list.append(re.search(r'先手 (.*?)<',str(title[1])).group(0)[3:][:-1].split(" ")[0].replace("　","")) #ok
      #black_rank_list.append(re.search(r'先手 (.*?)<',str(title[1])).group(0)[3:][:-1].split(" ")[1]) #ok  
        
      white_list.append(re.search(r'後手 (.*?)<',str(title[2])).group(0)[3:][:-1].split(" ")[0].replace("　","")) #ok
      #white_rank_list.append(re.search(r'後手 (.*?)<',str(title[2])).group(0)[3:][:-1].split(" ")[1]) #ok  
      
      senkei_list.append(re.search(r':(.*?)<',str(title[3])).group(0)[2:][:-1])
      teai_list.append(re.search(r':(.*?)<',str(title[4])).group(0)[2:][:-1])

      df = pd.DataFrame({
          "game_id":game_id_list
          ,"game_date":game_date_list
          ,"game_title":game_title_list
          ,"black":black_list
          #,"black_rank":black_rank_list
          ,"white":white_list
          #,"white_rank":white_rank_list
          ,"senkei":senkei_list
          ,"teai":teai_list
          ,"url":url_list
          ,"transaction":i

      })
  base_df = pd.concat([base_df,df],sort=False)
  
  base_df = base_df[["game_id","game_date","game_title","black","white","senkei","teai","url","transaction"]]
  base_df = base_df.reset_index(drop=True)
  base_df.to_csv("shogi_taikyoku.csv",encoding="utf-8",header=True,index=False)



In [0]:
base_df.drop_duplicates(subset="game_id").shape

(9033, 9)

In [0]:
base_df.shape

(9033, 9)

# ここからベースコード加工用

In [0]:
base_df.head()

,game_id,game_date,game_title,black,white,senkei,teai,url,transaction
0,d3db76663680c2ad70926eddd1b448fee647ecc7,2020-02-06,第78期順位戦Ｃ級２組９回戦,福崎文吾,佐藤紳哉,矢倉,平手,https://shogidb2.com/games/d3db76663680c2ad709...,1
1,bd0082021cb3c5a21a185be3e3cf50a0389106cc,2020-02-06,第78期順位戦Ｃ級２組９回戦,阿部光瑠,村田智弘,三間飛車,平手,https://shogidb2.com/games/bd0082021cb3c5a21a1...,1
2,ad23f852079cb9fbb0562b54d4dd1e4ec79c61a4,2020-02-06,第78期順位戦Ｃ級２組９回戦,村中秀史,大平武洋,矢倉,平手,https://shogidb2.com/games/ad23f852079cb9fbb05...,1
3,a280b927f7ec94585189902d0f7ecd45696c708b,2020-02-06,第78期順位戦Ｃ級２組９回戦,山本博志,矢倉規広,相振り飛車,平手,https://shogidb2.com/games/a280b927f7ec9458518...,1
4,d5fc0bf601b8c1c2c0a08dcf1e788c9a95f72eb7,2020-02-06,第78期順位戦Ｃ級２組９回戦,黒田尭之,井出隼平,四間飛車,平手,https://shogidb2.com/games/d5fc0bf601b8c1c2c0a...,1


In [0]:
base_df1 = base_df.copy()
base_df2 = base_df.copy()

base_df1["key"] = base_df1["black"]
base_df2["key"] = base_df2["white"]

transaction_df = pd.concat([base_df1, base_df2])
transaction_df.to_csv("shogi_taikyoku_tran.csv",encoding="utf-8",header=True,index=False)




In [0]:
base_df["url"][0]

'https://shogidb2.com/games/d3db76663680c2ad70926eddd1b448fee647ecc7'

In [0]:
driver = webdriver.Chrome('chromedriver',options=options)
URL = base_df["url"][0]
driver.get(URL)
#kif-export
driver.find_element_by_id('kif-export').click()
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
driver.quit()
#kifu-modal > div > div > div.modal-body > div > textarea
for n in soup.select("#kifu-modal > div > div > div.modal-body > div > textarea"):
  kifu = re.search(r'>((.|\s)*?)<',str(n)).group(0)[1:][:-1]
  print(kifu)

開始日時：2020/02/06 1:00:00
終了日時：2020/02/06 10:34:00
棋戦：順位戦
場所：関西将棋会館
持ち時間：6時間
消費時間：92▲182△311
手合割：平手
先手：福崎文吾 九段
後手：佐藤紳哉 七段
戦型：矢倉
手数----指手---------消費時間--
1 ７六歩(77) (00:00/00:00:00)
2 ８四歩(83) (00:00/00:00:00)
3 ６八銀(79) (00:00/00:00:00)
4 ３四歩(33) (00:00/00:00:00)
5 ７七銀(68) (00:00/00:00:00)
6 ６二銀(71) (00:00/00:00:00)
7 ２六歩(27) (00:00/00:00:00)
8 ４二銀(31) (00:00/00:00:00)
9 ２五歩(26) (00:00/00:00:00)
10 ３三銀(42) (00:00/00:00:00)
11 ４八銀(39) (00:00/00:00:00)
12 ３二金(41) (00:00/00:00:00)
13 ５八金(49) (00:00/00:00:00)
14 ４一玉(51) (00:00/00:00:00)
15 ５六歩(57) (00:00/00:00:00)
16 ５四歩(53) (00:00/00:00:00)
17 ６六歩(67) (00:00/00:00:00)
18 ３一角(22) (00:00/00:00:00)
19 ６七金(58) (00:00/00:00:00)
20 ７四歩(73) (00:00/00:00:00)
21 ７九角(88) (00:00/00:00:00)
22 ８五歩(84) (00:00/00:00:00)
23 ４六角(79) (00:00/00:00:00)
24 ６四角(31) (00:00/00:00:00)
25 ６八玉(59) (00:00/00:00:00)
26 ７三角(64) (00:00/00:00:00)
27 ７八玉(68) (00:00/00:00:00)
28 ６四歩(63) (00:00/00:00:00)
29 ３六歩(37) (00:00/00:00:00)
30 ５二金(61) (00:00/00:00:00)
31 ５七銀(48) (00:00/0

In [0]:
soup

<html lang="ja"><head><script async="" src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.JKCQ2Hvuo0E.O/m=gapi_iframes_style_bubble/exm=auth,ytsubscribe/rt=j/sv=1/d=1/ed=1/am=wQE/rs=AGLTcCOl3FsgYnTqVeIFUJJD2j4-pO09lQ/cb=gapi.loaded_2"></script><script src="https://www.googletagservices.com/activeview/js/current/osd.js?cb=%2Fr20100101"></script><script src="https://partner.googleadservices.com/gampad/cookie.js?domain=shogidb2.com&amp;callback=_gfp_s_&amp;client=ca-pub-1706171618035325"></script><script id="google_shimpl" src="https://pagead2.googlesyndication.com/pagead/js/r20200219/r20190131/show_ads_impl_fy2019.js"></script><script async="" src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.JKCQ2Hvuo0E.O/m=auth/exm=ytsubscribe/rt=j/sv=1/d=1/ed=1/am=wQE/rs=AGLTcCOl3FsgYnTqVeIFUJJD2j4-pO09lQ/cb=gapi.loaded_1"></script><script async="" src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.JKCQ2Hvuo0E.O/m=ytsubscribe/rt=j/sv=1/d=1/ed=1/am=wQ